# Übung: Zeugniskonferenzen

Am Ende eines Schuljahres gibt es Zeugniskonferenzen. 
Dort setzen sich alle Lehrer und Lehrerinnen einer Klasse zusammen und besprechen die Zeugnisnoten.
In der Schule stehen dafür $R$ Konferenzräume zur Verfügung. Jede Zeugniskonferenz dauert eine Stunde und muss zu einem von $T$ möglichen Terminen stattfinden.

Eine Lehrerin oder ein Lehrer unterrichtet üblicherweise in mehreren Klassen und muss daher an mehreren Konferenzen teilnehmen. Daher kommt es zu einem Konflikt, wenn zwei Konferenzen zum gleichen Zeitpunkt stattfinden, und beide eigentlich die Anwesenheit der gleichen Lehrkraft bedingen. Konflikte werden mittels der Matrix *conflicts* modelliert. Falls zwei Klassen $k_1$ und $k_2$ mindestens eine gemeinsamen Lehrkraft haben, so steht in der Matrix an der Stelle *conflicts[$k_1,k_2$]* eine 1 (und 0 sonst). Für eine Einteilung zählt jedes Paar an Klassen $k_1,k_2$, die mindestens eine gemeinsame Lehrkraft haben als ein Konflikt.

Ziel der Planung ist es nun eine Einteilung mit möglichst wenigen Konflikten zu finden. Ein Konflikt ist dabei ein (ungeordnetes) Paar $k_1,k_2$ von Konferenzen, die zur gleichen Zeit stattfinden sollen und beide die Teilnahme der gleichen Lehrkraft voraussetzen. (Warum "ungeordnet"? Wir wollen nicht $k_1,k_2$ **und** $k_2,k_1$ zählen -- dann hätten wir jeden Konflikt zweimal gezählt.)

**AUFGABE:** Implementieren Sie eine lokale Suche, die eine Einteilung mit möglichst wenigen Konflikten findet.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Methoden zum Generieren von Instanzen

Eine Instanz <code>inst</code> wird beschrieben durch folgende Parameter:
* <code>inst.R</code>: die Zahl der Räume
* <code>inst.T</code>: die Zahl der möglichen Termine
* <code>inst.K</code>: die Zahl der Konferenzen
* <code>inst.L</code>: die Zahl der Lehrer
* <code>inst.conflicts</code>: die Matrix der Konflikte

Um es ein wenig einfacher zu halten, nehmen wir an, dass die Zahl der Konferenzen so groß ist, dass zu jedem Termin jeder Raum genutzt werden muss, dass also $K=R\cdot T$ gilt. (Warum ist das einfacher? Weil wir uns dann nicht überlegen müssen, wie wir eine ungenutzte Termin/Raum-Kombination darstellen müssen.)

In [ ]:
class instance:
    def __init__(self,R,T,K,L,min_classes = 3, max_classes = 11):
        self.R = R
        self.T = T
        self.K = K
        self.L = L
        self.conflicts = generate_conflicts(self.K,self.L,min_classes,max_classes)

def generate_instance(R,T,K,L,seed=None):
    np.random.seed(seed)
    return instance(R,T,K,L)

def generate_conflicts(K,L,min_classes,max_classes):
    '''K Klassen, L Lehrer, min_classes/max_classes eines Lehrers'''
    conflicts = np.zeros((K,K))
    for l in range(L):
        number_of_classes = np.random.randint(min_classes,np.min([K,max_classes])+1)
        # Zufällige Reihenfolge, nur die ersten num_of_classes sind relevant
        order = np.random.permutation(K)
        for i in range(number_of_classes):
            for j in range(i):
                conflicts[order[i],order[j]] = 1
                conflicts[order[j],order[i]] = 1
    return conflicts

def get_instance():
    R = 5
    T = 20
    K = 100
    L = 50
    seed = 220
    inst = generate_instance(R,T,K,L,seed=seed)
    np.random.seed(None)
    return inst

## Visualisierungs- und Evaluationsmethoden

Wir stellen jede Einteilung (<code>schedule</code>) als Matrix (<code>array</code>) der Dimensionen $T\times R$ dar. Ein Eintrag von $k$ an der Stelle $t,r$ wird dann so interpretiert, dass die Konferenz mit Nummer $k$ zum Termin $t$ in Raum $r$ stattfindet. Die Methode <code>count_conflicts</code> zählt die Zahl der Konflikte und ist somit die Zielfunktion dieser Aufgabe. Die Methode <code>show_conflicts</code> dient der Visualisierung.

In [ ]:
def get_schedule_conflicts(inst,schedule):
    schedule=schedule.reshape((inst.T,inst.R))
    schedule_conflicts=np.zeros((inst.T,inst.R))
    for t in range(inst.T):
        at_same_time=schedule[t]
        # iterate over pairs of classes at time t
        for r,k1 in enumerate(at_same_time):
            for k2 in at_same_time:
                if k1==k2: # same class, no conflict
                    continue
                schedule_conflicts[t,r]+=inst.conflicts[k1,k2]
    return schedule_conflicts>0

def count_conflicts(inst,schedule):
    schedule_conflicts=get_schedule_conflicts(inst,schedule)
    return np.sum(schedule_conflicts)//2 # divide by 2 as every conflict was counted twice

def show_conflicts(inst,schedule):
    schedule_conflicts=get_schedule_conflicts(inst,schedule)    
    plt.xticks(range(inst.T))
    plt.xlabel('Zeit')
    plt.yticks(range(inst.R))
    plt.ylabel('Räume')
    plt.imshow(np.transpose(schedule_conflicts),cmap = 'bwr')
    plt.title('Konflikte in rot, Anzahl: {}'.format(count_conflicts(inst,schedule)))

Um ein Beispiel betrachten zu können, teilen wir die Konferenzen einmal nach Nummer ein, dh, Konferenz 0 findet zum Zeitpunkt 0 in Raum 0 statt, Konferenz 1 zum Zeitpunkt 0 in Raum 1, usw. 

In [ ]:
inst = get_instance()

## Ein komplett dämlicher Plan
numerical_schedule = np.arange(inst.K).reshape((inst.T,inst.R))

## Konflikte visualisieren
show_conflicts(inst,numerical_schedule)

Man beachte, dass die Zahl der roten Kästchen nicht mit der berichteten Anzahl überein stimmt. Das liegt daran, dass ein Kästchen rot ist, wenn die Konferenz im Konflikt mit einer anderen steht, wir für die Konfliktzahl aber die Zahl der Paare von Konflikt-Konferenzen zählen.

Bisher haben wir eine Zuteilung (<code>schedule</code>) als $T\times R$-Matrix aufgefasst und das macht auch Sinn. Für den folgenden Algorithmus ist es aber einfacher, wenn das <code>schedule</code> einfach nur 1-dimensionale Liste dargestellt wird. Dazu nehmen wir die Matrix und machen sie flach. Das geht so.

In [ ]:
## ein array zur Demonstration
A=np.arange(12).reshape(4,3)
A

Nun hauen wir <code>A</code> platt:

In [ ]:
L=A.flatten()
L

Und zurück:

In [ ]:
L.reshape(4,3)

Das bedeutet wir können ganz einfach eine zufällige Startlösung erzeugen.

In [ ]:
def get_random_schedule(inst):
    numerical_schedule=range(inst.K)
    schedule=np.random.permutation(numerical_schedule)
    return schedule

rnd_schedule=get_random_schedule(inst)
rnd_schedule

Wenn wir nun die Matrix-Form haben wollen (weil wir eben ablesen wollen, welche Konferenz zu welcher Zeit in welchem Raum ist), dann können wir die Matrix-Form mit <code>reshape</code> sehr einfach erhalten.

In [ ]:
rnd_schedule.reshape((inst.T,inst.R))

## Lokale Suche

Wir wollen nun einen Algorithmus der lokalen Suche implementieren. Wir beginnen mit einer zufälligen Permutation (siehe <code>get_random_schedule</code>). In jedem Schritt der lokalen Suche testen Sie

* je 10 mal Vertauschen zweier Einträge
* je 5 mal Verschieben eines Intervals zufälliger Größe an eine zufällige neue Position
* 1 komplett zufällige Permutation
* 1 lokale Modifikation Ihrer Wahl

Die ersten drei Methoden sind für Sie bereits implementiert. Eine kleine Subtilität: Allen der drei Methoden wird die Instanz <code>inst</code> als Parameter übergeben; keine der Methoden benutzt den Parameter. Warum also wird <code>inst</code> übergeben? Ich weiß nicht, was für eine vierte Methode Sie sich ausdenken, eventuell benötigt die Informationen über die Instanz. Um's einfacher zu halten, wollte ich, dass alle lokalen Schritte die gleiche Funktionssignatur haben und daher wird eben <code>inst</code> übergeben.

In [ ]:
def random_single_swap(inst,schedule):
    """Zufälliger Tausch von zwei Konferenzen"""
    schedule = schedule.copy() # don't change the original schedule
    k_1 = np.random.randint(len(schedule))
    k_2 = np.random.randint(len(schedule))
    schedule[k_1],schedule[k_2]=schedule[k_2],schedule[k_1]
    return schedule
    
def random_shift(inst,schedule): 
    """Zufälliger shift im schedule"""
    schedule = schedule.copy()
    old_pos = np.random.randint(len(schedule))
    length = np.random.randint(len(schedule)-old_pos)
    new_pos = np.random.randint(len(schedule)-length)
    sublist = schedule[old_pos:old_pos+length+1]
    schedule = np.delete(schedule,range(old_pos,old_pos+length+1))
    schedule = np.insert(schedule,new_pos,sublist)
    return schedule

def random_permutation(inst,schedule):
    """Zufällige Permutation"""
    return np.random.permutation(schedule)

## zur Demonstration
schedule=list(range(10))
random_shift(inst,schedule)

### Aufgabe: Benutzen Sie lokale Suche, um einen möglichst konfliktfreien Plan zu erstellen

Beginnen Sie mit einer zufälligen Permutation. In jedem Schritt der lokalen Suche testen Sie
* je 10 mal Vertauschen zweier Einträge -> <code>random_single_swap</code>
* je 5 mal Verschieben eines Intervals zufälliger Größe an eine zufällige neue Position -> <code>random_shift</code>
* 1 komplett zufällige Permutation -> <code>random_permutation</code>
* 1 lokale Modifikation Ihrer Wahl

Wählen Sie aus diesen 17 Möglichkeiten den besten Plan und führen diesen Schritt aus (auch wenn der Plan schlechter ist als der Ausgangspunkt).

Gehen Sie auf diese Art 300 Schritte und geben den besten Plan aus, den Sie während der gesamten Suche gefunden haben. Geben Sie aus, wie viele Konflikte am Ende bestehen.

In [ ]:
### Ihr Code hier ###
